# Этапы построения модели

## 1. Структура

### 1.1. Sequential

В большинстве случаев ваша нейронная сеть -- _последовательность_ слоев. То есть они идут последовательно друг за другом, и выход каждого  слоя является входом к следующему.

Для описания подобного рода моделей в keras есть удобная модель: __Sequential__. Она автоматически соединяет слои правильно, как и описано выше.

Для большинства случаев на практике такой нам и будет достаточно.

```python
from keras.models import Sequential

model = Sequential([
    
    Dense(),
    Activation(),
    Dropout(),
    Dense(),
    # ...
    Dense()
    
])
```

### 1.2. Functional API

Если рассматривать каждый слой отдельно от всех других, то у него, конечно, есть свой вход и свой выход. Keras functional API, или, говоря по-русски, функциональный стиль описания моделей keras, позволяет нам описывать слои по отдельности и соединять в любом порядке, соединять несколько выходов и один и все такое.

```python
from keras.models import Model

input_layer = Input()           # вход
dense1 = Dense()(input_layer)   # полносвязный слой подсоединяем к входу
dense2 = Dense()(dense1)        # следующий полносвязный - к предыдущему
output_layer = Dense()(dense2)  # выходной слой

model = Model(inputs=input_layer, outputs=output_layer)
```

## 2. Слои

| __Как называется по-русски__ | __Как называется по-пацански__ | __Как называется в мире keras__ | __Что делает__ |
|:--------------------:|:------------------------:|:----------------------:|
| Полносвязный | Дэнс | __Dense__ | $\sigma(\omega x + b)$ - то есть предоставляет и хранит матрицу весов $\omega$ и вектор смещений $b$, являющимися обучаемыми параметрами. Опционально может сразу применить функцию активации $\sigma$ от результата _(обычно так и делают)_.|
| Активация       | Активэйшн    |       __Activation__              | Только функция активации $\sigma$. |
| Нормализация батча | Батчнорм | __BatchNormalization__ | Модифицирует выходные значения очередного слоя так, чтобы среднее значение активаций было равным 0 и среднеквадратичное отклонение было равным 1 |
| Исключение | Дропаут | __Dropout__ | Случайным образом выкидывает некоторое количество весов, чтобы нейросеть училась формировать предсказания на том, что осталось. Таким образом она будет учиться лучше генерализовать и меньше запоминать выборку, словом, является регуляризатором. |

## 3. Функция потерь

### 3.1. Для задачи регрессии

#### 3.1.1. MSE

$$ MSE = \frac{1}{N}\sum_{i=1}^{N} (y_i - \hat{y}_i)^2 $$

In [ ]:
model.compile(loss='mean_squared_error')

#### 3.1.2. MAE

$$ MAE = \frac{1}{N}\sum_{i=1}^{N} |y_i - \hat{y}_i| $$

In [ ]:
model.compile(loss='mean_absolute_error')

__У вас в данных есть выбросы?__ $\rightarrow$ Используй __MAE__!

__Уверен, что они выбросы?__ $\rightarrow$ Используй __MAE__!

__"Выбросы" - нормальные значения, которые могут появляться?__ $\rightarrow$ Используй __MSE__!

### 3.2. Для задачи классификации

#### 3.2.1. Бинарная кросс-энтропия

$$ BinaryCrossEntropy = −(y\log(\hat{y}) + (1 − y)\log(1 − \hat{y})) $$

In [ ]:
model.compile(loss='binary_crossentropy')

#### 3.2.2. Категориальная кросс-энтропия

$$ CategoricalCrossEntropy = −\sum_{c=1}^{M} y_{o,c}\log(p_{o, c}) $$

При $M=2$ вырождается в бинарную, очевидно.

In [ ]:
model.compile(loss='categorical_crossentropy')

## 4. Метрики качества

Метрику очень важно не путать с функцией потерь!

__Метрика__ нужна для _человека_, чтобы измерить качество модели в понятной ему величине. __Функция потерь__ нужна для _алгоритма_, чтобы правильно ползать по поверхности функции стоимости и находить лучшее решение. 

Эти две величины __не обязательно__ напрямую связаны! Достижение лучшего решение по функции стоимости не означает, что в результате вы получите самую точную модель.

_P.s. Но в целом в большинстве случаев так и происходит. Кроме того, в случае регрессии, например, обычно метрики и функции потерь совпадают, скажем, везде берут MSE._

### 4.1. Для задачи регрессии

#### 4.1.1. MSE, MAE

Точно так же, как и выше.

#### 4.1.2. RMSE

Допустим, мы предсказываем сумму продаж в рублях. Тогда MSE даст нам.. квадрат рублей? Не очень удобно. То ли дело взять из значения ошибки корень, то есть вычислить __RMSE__ - Root Mean Squared Error.

In [ ]:
import keras.backend as K
from keras.metrics import mean_squared_error

def rmse(y_true, y_pred):
    return K.sqrt(mean_squared_error(y_true, y_pred))

model.compile(metrics=[rmse])

### 4.2. Для задачи классификации

#### 4.2.1. Accuracy

Доля правильно угаданных экземляров выборки, ничего особенного.

In [ ]:
model.compile(metrics=['accuracy'])

# Почитать на дом

https://habr.com/company/ods/blog/328372/